In [1]:
using LinearAlgebra
using ControlSystems
using Polynomials

# https://juliamath.github.io/Polynomials.jl/stable/
# https://docs.julialang.org/en/v1/manual/complex-and-rational-numbers/

In [2]:
fromroots([-7, -8, -9])

Polynomial(504 + 191*x + 24*x^2 + x^3)

In [3]:
A = [-24 1 0; -191 0 1; -504 0 0]
B = [0; 1; 6]
C = [1 0 0]

1×3 Matrix{Int64}:
 1  0  0

In [4]:
Om = [C; C*A; C*A^2]
Om

3×3 Matrix{Int64}:
   1    0  0
 -24    1  0
 385  -24  1

In [5]:
rank(Om)

3

In [6]:
obsv(A,C)

3×3 Matrix{Int64}:
   1    0  0
 -24    1  0
 385  -24  1

In [7]:
# From Percentage Overshoot to Damping Ratio
os = 20;
ζ = -log(os/100)/sqrt( π^2 + log(os/100)^2 )

0.4559498107691261

In [8]:
# From Settling Time and Damping Ratio to Natural Frequency
Ts = 2;
# ω_n = -log(0.02*sqrt(1-ζ^2)) / (ζ * Ts);
ω_n = -log(0.02*sqrt(1-ζ^2)) / (ζ * Ts)# No Approximaition

4.417756690959335

In [9]:
# Desired Pole Location Based on Transient Requirement
pole_loc = roots( Polynomial([ω_n^2, 2*ζ*ω_n, 1]) )

2-element Vector{ComplexF64}:
 -2.0142753272669496 - 3.931827703051037im
 -2.0142753272669496 + 3.931827703051037im

In [10]:
# Observe needs to be N times faster than the closed loop response
N = 10
pole_obvs = pole_loc*N

2-element Vector{ComplexF64}:
 -20.142753272669495 - 39.31827703051037im
 -20.142753272669495 + 39.31827703051037im

In [20]:
pole_choice = N*real(pole_loc[1]) # 10 times faster than the real part of the dominant second order pole
pole_choice_obvs = pole_choice*N
eqn_char_desired = fromroots([pole_choice_obvs,pole_obvs[1],pole_obvs[2]])

Polynomial(393117.5384458851 + 10066.267606124351*x + 241.71303927203394*x^2 + 1.0*x^3)

In [25]:
L = A[:,1]+[eqn_char_desired[2]; eqn_char_desired[1]; eqn_char_desired[0]]

3-element Vector{Float64}:
    217.71303927203394
   9875.267606124351
 392613.5384458851

In [26]:
A-L*C

3×3 Matrix{Float64}:
   -241.713      1.0  0.0
 -10066.3        0.0  1.0
     -3.93118e5  0.0  0.0

## Simulate a ramp response with non-zero initial error

In [34]:
using ControlSystems
using LinearAlgebra: I
using Plots

A = A
Bo = [0;0;0]
C = C
L = L
sys = ss(A-L*C,Bo,C,0)

StateSpace{Continuous, Float64}
A = 
    -241.71303927203394  1.0  0.0
  -10066.267606124351    0.0  1.0
 -393117.5384458851      0.0  0.0
B = 
 0.0
 0.0
 0.0
C = 
 1.0  0.0  0.0
D = 
 0.0

Continuous-time state-space model

In [74]:
u(x,t) = [t]
t  = 0:0.01:0.5
# u(x,t) = t
x0 = [1;0;0]
y, t, x, uout = lsim(sys, u, t, x0=x0)
plot(t,y)#, lab=["Position" "Velocity"], xlabel="Time [s]")

LoadError: Expects 51 elements in each col of y, found 1.

In [75]:
y

1×51 Matrix{Float64}:
 1.0  -0.0402167  -0.157638  -0.134955  …  -1.26269e-5  -1.13072e-5

In [72]:
u = None

LoadError: UndefVarError: `None` not defined

In [60]:
u(x,t) =  t

u (generic function with 2 methods)

In [63]:
range(0,stop=0.5,length=200)

0.0:0.002512562814070352:0.5

In [49]:
u[1]

LoadError: MethodError: no method matching getindex(::typeof(u), ::Int64)